In [4]:
import yaml
import operator
import math
import re
from typing import List

ops = {
    '+' : operator.add,
    '-' : operator.sub,
    '*' : operator.mul,
    '/' : operator.truediv,  # use operator.div for Python 2
    '%' : operator.mod,
    '^' : operator.xor,
}

class Item:
    def __init__(self, worry_level: int):
        self.worry_level = worry_level
        
    def __str__(self):
        return str(self.worry_level)
    
    def __repr__(self):
        return self.__str__()

class Monkey:
    def __init__(self, name, data):
        self.name = name
        self.divisor = int(data['Test'].split(' ')[-1])


        items = [data['Starting items']] if isinstance(data['Starting items'], int) else data['Starting items'].split(',')
        self.items = [Item(int(worry_level)) for worry_level in items]
        
        self.operation = data['Operation']
        self.true_monkey = int(data['If true'].split(' ')[-1])
        self.false_monkey = int(data['If false'].split(' ')[-1])
        
        
        self.count = 0
        
    def throw(self, item, monkeys):
        if item.worry_level % self.divisor == 0:
            monkeys[self.true_monkey].items.append(item)
        else:
            monkeys[self.false_monkey].items.append(item)
            
    def inspect(self, item, monkeys):
        gcm = math.prod([monkey.divisor for monkey in monkeys])
        
        result = re.search(r"new = ([a-z0-9]+) ([\*\+\-\%]) ([a-z0-9]+)", self.operation)
        
        l, op, r = result.groups()
        
        left = item.worry_level if l == 'old' else int(l)
        right = item.worry_level if r == 'old' else int(r)
        
        item.worry_level = ops[op](left, right) % gcm
        
        self.count += 1
        
    def __str__(self):
        return f'{self.name} ({self.count}) - {self.items} - {self.operation} - {self.divisor}'
    
    def __repr__(self):
        return self.__str__()
    
class Barrel:
    def __init__(self):
        self.monkeys: List[Monkey] = []
        
    def add(self, monkey):
        self.monkeys.append(monkey)
        
    def get_common_multiple(self):
        return math.prod([monkey.divisor for monkey in self.monkeys])
    
    def simulate(self, rounds):
        for i in range(rounds):
            for monkey in self.monkeys:
                while monkey.items:
                    item = monkey.items.pop(0)
                    monkey.inspect(item, self.monkeys)
                    monkey.throw(item, self.monkeys)
        
    def get_monkey_business_level(self):
        monkeys = sorted(self.monkeys, key=lambda monkey: -monkey.count)

                    
        return monkeys[0].count * monkeys[1].count

with open('../inputs/11.txt') as f:
    monkey_to_data = yaml.safe_load(f)
    
    barrel = Barrel()
    for key, data in monkey_to_data.items():
        name = int(key.split(' ')[-1])
        barrel.add(Monkey(name, data))
                
    barrel.simulate(10000)
    
    print(barrel.get_monkey_business_level())
    

32059801242
